In [9]:
import copy
import pandas as pd
import numpy as np
import librosa
import seaborn as sb
import matplotlib.pyplot as plt
import itertools
import re
import random
import gc
import math
import os
from sklearn.utils import shuffle
from operator import itemgetter, attrgetter, methodcaller
from os import listdir
from os.path import isfile, join
from numpy import median, diff
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Dropout, BatchNormalization
from sklearn.cross_validation import train_test_split, cross_val_score

# Instead of training models for each note
- decide which will have notes, for now pick x with one, y with another (train model for this + holds, hands, mines, rolls etc later)
- for each note that will have something, decide what combo it has (train from prev notes (not all 48, pick more relevant ones)) (4 for one note, 6 for 2, 4 for 3) and pick highest class

### TODOS
- try to predict hold/roll there (would need to train on later beat info as well)
- generate percent of single double notes etc with a nn

## Classes
- 1: one note
- 2: two notes
- 3: three or four notes
- 4: hold start
- 5: roll start
- 6: mine

In [4]:
steps_per_bar = 48
class SongFile:
    def __init__(self, key, folder, stepfile, music_file):
        misc = pd.read_csv('data/{0}_misc.csv'.format(key)).values
        self.note_classes = pd.read_csv('generated_data/{0}_note_classes_generated.csv'.format(key), converters={'0': lambda x: float(x)}).values
        self.notes = pd.read_csv('data/{0}_notes.csv'.format(key), converters={'0': lambda x: str(x)}).values
        self.folder = folder
        self.name = key.split('~')[1]
        self.music_name = music_file
        self.stepfile_name = stepfile
        self.offset = misc[0][0]
        self.beat_length = 60. / misc[1][0]
        self.bpm = misc[1][0]
        self.extension = music_file.split('.')[1]

In [5]:
songs_to_use = pd.read_csv('data/songs_to_use.csv').values
save_files = listdir('data')
save_files_generated = listdir('generated_data')
songs = {}
for song_data in songs_to_use:
    key = song_data[0]
    if '{0}_misc.csv'.format(key) in save_files and '{0}_note_classes_generated.csv'.format(key) in save_files_generated:
        songs[key] = SongFile(key, song_data[1], song_data[2], song_data[3])

In [6]:
beats_to_track = 48
num_classes_one_note = 4
num_classes_two_note = 6
class_map_one_note = {
    '1000': 0,
    '0100': 1,
    '0010': 2,
    '0001': 3
}
class_reverse_map_one_note = ['1000', '0100', '0010', '0001']

class_map_two_note = {
    '1001': 0,
    '0110': 1,
    '1100': 2,
    '1010': 3,
    '0101': 4,
    '0011': 5
}
class_reverse_map_two_note = ['1001', '0110', '1100', '1010', '0101', '0011']

note_types = ['0', '1', 'M', '2', '4', '3']

def get_features_for_row(row):
    return [int(char == target) for target in note_types for char in row]

empty_row = [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
def get_previous_notes(index, features):
    previous_notes = [features[i] for i in range(index, index + song_padding) if not np.array_equal(features[i], empty_row)]
    return [empty_row] * (8 - len(previous_notes)) + previous_notes[-8:]
    
song_padding = beats_to_track * 2
song_end_padding = beats_to_track * 2
important_indices = [1, 2, 3, 4, 8, 16, 20, 6, 12, 18, 24, 30, 36, 42, 48, 54, 60, 66, 72, 78, 84, 90, 96]
important_indices_classes = [-96, -84, -72, -60, -48, -36, -24, -12, 0, 1, 2, 3, 4, 8, 16, 20, 6, 12, 18, 24, 30, 36, 42, 48, 54, 60, 66, 72, 78, 84, 90, 96]
def get_features(index, features, note_classes):
    indices = [index + song_padding - i for i in important_indices]
    indices_classes = [index + song_padding - i for i in important_indices_classes]
    past_classes = np.array([note_classes[i] for i in indices_classes]).flatten()
    past_features = np.array([features[i] for i in indices]).flatten()
    previous_notes = np.array(get_previous_notes(index, features)).flatten()
    return np.concatenate((past_classes, past_features, previous_notes), axis = 0)

In [7]:
def get_model_class_for_notes(row):
    note_counts = [row.count(note_type) for note_type in note_types]
    (blank, steps, mines, hold_starts, roll_starts, hold_ends) = note_counts
    
    model_classes = []
    if steps + hold_starts + roll_starts == 1:
        model_classes.append(1)

    if steps + hold_starts + roll_starts == 2:
        model_classes.append(2)
        
    if steps + hold_starts + roll_starts > 2:
        model_classes.append(3)
        
    if hold_starts > 0:
        model_classes.append(4)
        
    if roll_starts > 0:
        model_classes.append(5)
        
    if mines > 0:
        model_classes.append(6)
        
    return model_classes

def get_model_output_for_class(model_class, row):
    if model_class == 1 or model_class == 2 or model_class == 3:
        return [int(char == '1' or char == '2' or char == '4') for char in row]
    if model_class == 4:
        return [int(char == '2') for char in row]
    if model_class == 5:
        return [int(char == '4') for char in row]
    if model_class == 6:
        return [int(char == 'M') for char in row]

def get_hold_length(notes, note_row, note_column):
    i = 0
    while i < len(notes) - note_row:
        if notes[note_row + i][0][note_column] == '3':
            return i
        i += 1
    return False

In [28]:
hold_X = []
roll_X = []
hold_y = []
roll_y = []
X = [[] for i in range(7)]
y = [[] for i in range(7)]
for key in list(songs.keys()):
    note_classes = np.concatenate((([[1, 0, 0, 0, 0, 0, 0]] * song_padding), songs[key].note_classes, ([[1, 0, 0, 0, 0, 0, 0]] * song_end_padding)), axis = 0)
    notes = np.concatenate((([['0000']] * song_padding), songs[key].notes), axis = 0)
    if abs(len(note_classes) - len(notes) > 250):
        print ('Lengths dont match for {0}'.format(key))
        print ('{0} vs {1}'.format(len(note_classes), len(notes)))
        continue
    length = min(len(note_classes) - song_padding - song_end_padding, len(notes) - song_padding)
    features = np.array([get_features_for_row(notes[i][0]) for i in range(0, length + song_padding)])
    for i in range(length):
        row = notes[i + song_padding][0]
        model_classes = get_model_class_for_notes(row)
        for model_class in model_classes:
            X_row = get_features(i, features, note_classes)
            X[model_class].append(X_row)
            y[model_class].append(get_model_output_for_class(model_class, row))
            if model_class == 4:
                for j in range(4):
                    if row[j] == '2':
                        length = get_hold_length(notes, i + song_padding, j)
                        if length:
                            hold_X.append(X_row)
                            hold_y.append(length)
            if model_class == 5:
                for j in range(4):
                    if row[j] == '4':
                        length = get_hold_length(notes, i + song_padding, j)
                        if length:
                            roll_X.append(X_row)
                            roll_y.append(length)

X = [np.array(X_for_class) for X_for_class in X]
y = [np.array(y_for_class) for y_for_class in y]
hold_X = np.array(hold_X)
roll_X = np.array(roll_X)
for i in range(1,7):
    X[i], y[i] = shuffle(X[i], y[i], random_state=0)
hold_X, hold_y = shuffle(hold_X, hold_y, random_state=0)
roll_X, roll_y = shuffle(roll_X, roll_y, random_state=0)

In [12]:
hold_lengths = [3, 6, 9, 12, 18, 24, 36, 48]
def get_closest_hold_lengths(lengths):
    closest = [np.argmax([-abs(length - aprox) for aprox in hold_lengths]) for length in lengths]
    closest_one_hot = np.zeros((len(closest), len(hold_lengths)))
    closest_one_hot[np.arange(len(closest)), closest] = 1
    return np.array(closest_one_hot)

In [13]:
hold_y_transformed = get_closest_hold_lengths(hold_y)
X_hold_train, X_hold_test, y_hold_train, y_hold_test = train_test_split(hold_X, hold_y_transformed, test_size=0.2, random_state=42)

Train on 5487 samples, validate on 1372 samples
Epoch 1/2
5487/5487 [==============================] - 5s - loss: 1.7789 - acc: 0.3845 - val_loss: 2.0513 - val_acc: 0.2770
Epoch 2/2
5487/5487 [==============================] - 4s - loss: 1.3003 - acc: 0.5847 - val_loss: 1.7163 - val_acc: 0.4125


In [15]:
roll_y_transformed = get_closest_hold_lengths(roll_y)
X_roll_train, X_roll_test, y_roll_train, y_roll_test = train_test_split(roll_X, roll_y_transformed, test_size=0.2, random_state=42)

In [20]:
hold_model = Sequential()

hold_model.add(Dense(512, input_shape=(968,)))
hold_model.add(BatchNormalization())
hold_model.add(Activation('relu'))
hold_model.add(Dropout(0.5))

hold_model.add(Dense(512))
hold_model.add(BatchNormalization())
hold_model.add(Activation('relu'))
hold_model.add(Dropout(0.5))

hold_model.add(Dense(len(hold_lengths)))
hold_model.add(BatchNormalization())
hold_model.add(Activation('softmax'))

hold_model.compile(loss='categorical_crossentropy',
              optimizer='adagrad',
              metrics=['accuracy'])

hold_model.fit(X_hold_train, y_hold_train, nb_epoch=20, batch_size=64, verbose=1, validation_data=(X_hold_test, y_hold_test))
hold_model.save('models/hold_length_model.h5')

Train on 5487 samples, validate on 1372 samples
Epoch 1/20
5487/5487 [==============================] - 1s - loss: 1.8947 - acc: 0.3231 - val_loss: 1.9650 - val_acc: 0.3120
Epoch 2/20
5487/5487 [==============================] - 1s - loss: 1.5705 - acc: 0.4680 - val_loss: 1.7886 - val_acc: 0.3972
Epoch 3/20
5487/5487 [==============================] - 1s - loss: 1.4374 - acc: 0.5181 - val_loss: 1.6230 - val_acc: 0.4687
Epoch 4/20
5487/5487 [==============================] - 1s - loss: 1.3349 - acc: 0.5688 - val_loss: 1.4928 - val_acc: 0.5153
Epoch 5/20
5487/5487 [==============================] - 1s - loss: 1.2626 - acc: 0.5925 - val_loss: 1.4669 - val_acc: 0.5270
Epoch 6/20
5487/5487 [==============================] - 1s - loss: 1.1842 - acc: 0.6337 - val_loss: 1.4217 - val_acc: 0.5270
Epoch 7/20
5487/5487 [==============================] - 1s - loss: 1.1184 - acc: 0.6586 - val_loss: 1.4141 - val_acc: 0.5299
Epoch 8/20
5487/5487 [==============================] - 1s - loss: 1.0618 - a

In [22]:
roll_model = Sequential()

roll_model.add(Dense(512, input_shape=(968,)))
roll_model.add(BatchNormalization())
roll_model.add(Activation('relu'))
roll_model.add(Dropout(0.5))

roll_model.add(Dense(512))
roll_model.add(BatchNormalization())
roll_model.add(Activation('relu'))
roll_model.add(Dropout(0.5))

roll_model.add(Dense(len(hold_lengths)))
roll_model.add(BatchNormalization())
roll_model.add(Activation('softmax'))

roll_model.compile(loss='categorical_crossentropy',
              optimizer='adagrad',
              metrics=['accuracy'])

roll_model.fit(X_roll_train, y_roll_train, nb_epoch=20, batch_size=64, verbose=1, validation_data=(X_roll_test, y_roll_test))
roll_model.save('models/roll_length_model.h5')

Train on 189 samples, validate on 48 samples
Epoch 1/20
189/189 [==============================] - 0s - loss: 2.1933 - acc: 0.1958 - val_loss: 1.8109 - val_acc: 0.3333
Epoch 2/20
189/189 [==============================] - 0s - loss: 1.2344 - acc: 0.6190 - val_loss: 1.8288 - val_acc: 0.3125
Epoch 3/20
189/189 [==============================] - 0s - loss: 0.9441 - acc: 0.8095 - val_loss: 1.8253 - val_acc: 0.3542
Epoch 4/20
189/189 [==============================] - 0s - loss: 0.8070 - acc: 0.8942 - val_loss: 1.8261 - val_acc: 0.3958
Epoch 5/20
189/189 [==============================] - 0s - loss: 0.7634 - acc: 0.9259 - val_loss: 1.8525 - val_acc: 0.3750
Epoch 6/20
189/189 [==============================] - 0s - loss: 0.7093 - acc: 0.9312 - val_loss: 1.8715 - val_acc: 0.3958
Epoch 7/20
189/189 [==============================] - 0s - loss: 0.6410 - acc: 0.9735 - val_loss: 1.8690 - val_acc: 0.3750
Epoch 8/20
189/189 [==============================] - 0s - loss: 0.6263 - acc: 0.9577 - val_lo

In [23]:
def build_model(num_classes):
    model = Sequential()

    model.add(Dense(1024, input_shape=(968,)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.2))

    model.add(Dense(1024))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.2))

    model.add(Dense(num_classes))
    model.add(BatchNormalization())
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy',
                  optimizer='adagrad',
                  metrics=['accuracy'])
    
    return model

In [30]:
models = []
models.append(None)
for i in range(1, 7):
    model = build_model(4)
    X_train, X_test, y_train, y_test = train_test_split(X[i], y[i], test_size=0.2, random_state=42)
    model.fit(X_train, y_train, nb_epoch=10, batch_size=64, verbose=1, validation_data=(X_test, y_test))
    model.save('models/note_model_{0}.h5'.format(i))
    models.append(model)

Train on 66166 samples, validate on 16542 samples
Epoch 1/10
66166/66166 [==============================] - 52s - loss: 1.1968 - acc: 0.4240 - val_loss: 1.1680 - val_acc: 0.4379
Epoch 2/10
66166/66166 [==============================] - 50s - loss: 1.1235 - acc: 0.4738 - val_loss: 1.1559 - val_acc: 0.4518
Epoch 3/10
66166/66166 [==============================] - 48s - loss: 1.0790 - acc: 0.5048 - val_loss: 1.1545 - val_acc: 0.4525
Epoch 4/10
66166/66166 [==============================] - 43s - loss: 1.0361 - acc: 0.5329 - val_loss: 1.1586 - val_acc: 0.4565
Epoch 5/10
66166/66166 [==============================] - 43s - loss: 0.9926 - acc: 0.5613 - val_loss: 1.1837 - val_acc: 0.4411
Epoch 6/10
66166/66166 [==============================] - 50s - loss: 0.9468 - acc: 0.5881 - val_loss: 1.1874 - val_acc: 0.4547
Epoch 7/10
66166/66166 [==============================] - 62s - loss: 0.9106 - acc: 0.6092 - val_loss: 1.1893 - val_acc: 0.4590
Epoch 8/10
66166/66166 [==============================